### Diff Analysis

Comparo los resultados de los grafos. Tomo como grafo a comparar, para cada contrato, el construido usando VeriSol con un txBound = 8. Comparo, contra ese, los grafos construidos usando Echidna.

Los resultados serán del tipo tupla (n, e), donde n y e serán la diferencia de nodos y ejes respectivamente de cada grafo con respecto al grafo base. Esos valores de n y e pueden ser tanto negativos (encontramos menos de los que se había encontrado) o positivos (encontramos más cosas).


In [1]:
import os
import json5 as json
import pygraphviz as pgv

In [2]:

benchmark_1 = [
    # Original
    "AssetTransfer",
    "BasicProvenance",
    "DefectiveComponentCounter",
    "DigitalLocker",
    "FrequentFlyerRewardsCalculator",
    "HelloBlockchain",
    "RefrigeratedTransportation",
    "RoomThermostat",
    "SimpleMarketplace",
    # Fixed
    "AssetTransferFixed",
    "BasicProvenanceFixed",
    "DefectiveComponentCounterFixed",
    "DigitalLockerFixed",
    "HelloBlockchainFixed",
    "RefrigeratedTransportationFixed", 
    "SimpleMarketplaceFixed",
]

ignore_1 = [
    ("AssetTransfer", "epa"),
    ("DigitalLocker", "epa"),
    ("AssetTransferFixed", "epa"),
    ("DigitalLockerFixed", "epa"),
]

benchmark_2 = [
    # Original
    "Auction",
    "Crowdfunding",
    "EPXCrowdsale",
    "EscrowVault",
    "RefundEscrow",
    "RockPaperScissors",
    "SimpleAuction",
    "ValidatorAuction",
    # Variations
    "RefundEscrowWithdraw", # sólo e
    "EPXCrowdsaleIsCrowdsaleClosed",  # e
    "CrowdfundingTime_Base", # e
    "CrowdfundingTime_BaseBalance", # e
    "CrowdfundingTime_BaseBalanceFix", # e
    "ValidatorAuction", # S
    # "ValidatorAuction_withdraw", # e
    "SimpleAuctionTime", # e
    "SimpleAuctionEnded", # e
    "SimpleAuctionHB", # S
    "AuctionEnded", # e
    "AuctionWithdraw", # e
]


ignore_2 = [
    ("Auction", "states"),  # Tiene el problema de la variable State y StateType, que no existen en el contrato.
    ("SimpleAuction", "states"), # highestBidderA, pendingReturnsArray no existen.
    ("ValidatorAuction", "epa"),  # Demoró horas con test limit 100.
    
    # Ignoro los siguientes porque así estaba en el benchmark_info de SolidityAbstractor
    ("RefundEscrowWithdraw", "states"),
    ("EPXCrowdsaleIsCrowdsaleClosed",  "states"),
    ("CrowdfundingTime_Base", "states"),
    ("CrowdfundingTime_BaseBalance", "states"),
    ("CrowdfundingTime_BaseBalanceFix", "states"),
    ("ValidatorAuction", "epa"),
    ("ValidatorAuction", "states"),
    ("ValidatorAuction_withdraw", "states"),
    ("SimpleAuctionTime", "states"),
    ("SimpleAuctionEnded", "states"),
    ("SimpleAuctionHB", "epa"),
    ("AuctionEnded", "states"),
    ("AuctionWithdraw", "states"),
]

In [3]:
class MyGraph:
    def __init__(self, graph):
        self.graph = pgv.AGraph(graph)
        self.nodes = set(self.graph.nodes())
        self.edges = self.get_edges_with_labels()
        self.labels_of_nodes = self.get_labels_of_nodes()

    def get_edges_with_labels(self):
        return set(map(lambda x: (x, x.attr["label"]), self.graph.edges()))
    
    def get_labels_of_nodes(self):
        return set(map(lambda x: x.attr["label"], self.graph.nodes()))
    
    def is_empty(self):
        return len(self.nodes) + len(self.edges) == 0

    def has_dummy_transitions(self):
        # iterate through its edges and check if they have the string "dummy" in their label
        return any(map(lambda x: "dummy" in x[1], self.edges))

### Sanity Checks

- Ver que ningún grafo esté vacío
- Ver si hay grafos con transiciones dummy

In [ ]:
def fix_graph_name(contract, dir_name):
    previous_name = contract 
    graph_file_name = contract[:-5] if contract.endswith("Fixed") else contract
    graph_file_name = "SimpleAuction" if dir_name.startswith("SimpleAuction") else graph_file_name
    graph_file_name = "RefundEscrow" if dir_name.startswith("RefundEscrow") else graph_file_name
    graph_file_name = "EPXCrowdsale" if dir_name.startswith("EPXCrowdsale") else graph_file_name
    graph_file_name = graph_file_name.replace("Time", "T")
    if dir_name.startswith("CrowdfundingT_BaseBalance"):
        graph_file_name = "CrowdfundingBase"
    if dir_name.startswith("Auction"):
        graph_file_name = "Auction"
    return graph_file_name

In [4]:
ECHIDNA_DIR = "../results/echidna_output"
VERISOL_DIR = "../results/verisol_output"
ORIGINAL_VERISOL_DIR = "../graph"

contracts_to_compare_1 = [(c, mode) for mode in ['epa', 'states'] for c in benchmark_1 if (c, mode) not in ignore_1]
contracts_to_compare_2 = [(c, mode) for mode in ['epa', 'states'] for c in benchmark_2 if (c, mode) not in ignore_2]

test_limits = [1_000, 50_000, 500_000]
tx_bounds = [4, 8]

empty_count = 0
dummy_count = 0
contracts_to_compare = contracts_to_compare_1 + contracts_to_compare_2
options = ["not_reduce_combinations", "reduce_combinations"]
for contract_and_mode in contracts_to_compare:
    contract = contract_and_mode[0]
    mode = contract_and_mode[1]
    for option in options:
        if option == "reduce_combinations" and mode == "states":
            continue
        for test_limit in test_limits:
            dir_name = f"{contract[:-5]}_fixed" if contract.endswith("Fixed") else contract
            dir_name = dir_name.replace("Time", "T")
            graph_file_name = fix_graph_name(contract, dir_name)
            echidna_graph = f"{ECHIDNA_DIR}/{dir_name}/{mode}/{test_limit}/{option}/graph/{graph_file_name}_Mode.{mode}"
            # print(echidna_graph)
            my_echidna_graph = MyGraph(echidna_graph)
            if my_echidna_graph.is_empty():
                empty_count += 1
                print(f"{empty_count}. EMPTY ERROR: Empty graph in {echidna_graph}")
            elif my_echidna_graph.has_dummy_transitions():
                dummy_count += 1
                print(f"{dummy_count}. DUMMY ERROR: Dummy transitions in {echidna_graph}")

print(f"We found {empty_count} empty graphs.")
print(f"We found {dummy_count} graphs with one or more dummy transitions.")

NameError: name 'fix_graph_name' is not defined

In [5]:
# Given two graphs, check some conditions
# If they have the same amount of nodes and edges, check if they are different
# If they have different amount of nodes, check if the one with less nodes has a node that is not in the other
# If they have different amount of edges, check if the one with less edges has an edge that is not in the other
# If some of that is true, return True, else return False
def check_graphs(graph_1, graph_2):
    if len(graph_1.nodes) == len(graph_2.nodes):
        if graph_1.nodes != graph_2.nodes:
            print("1. Tienen la misma cantidad de nodos pero no son iguales")
            print(graph_1.nodes - graph_2.nodes)
            print(graph_2.nodes - graph_1.nodes)
            return True
    if len(graph_1.edges) == len(graph_2.edges):
        if graph_1.edges != graph_2.edges:
            print("2. Tienen la misma cantidad de aristas pero no son iguales")
            print(graph_1.edges - graph_2.edges)
            print(graph_2.edges - graph_1.edges)
            return True
    if len(graph_1.nodes) < len(graph_2.nodes):
        if not graph_1.nodes.issubset(graph_2.nodes):
            print("3. El grafo 1 tiene menos nodos que el grafo 2, pero tiene nodos que el grafo 2 no tiene")
            print(graph_1.nodes - graph_2.nodes)
            print(graph_2.nodes - graph_1.nodes)
            return True  
    if len(graph_1.edges) < len(graph_2.edges):
        if not graph_1.edges.issubset(graph_2.edges):
            print("4. El grafo 1 tiene menos aristas que el grafo 2, pero tiene aristas que el grafo 2 no tiene")
            print(graph_1.edges - graph_2.edges)
            print(graph_2.edges - graph_1.edges)
            return True
    else:
        return False

In [6]:
def diff_verisol(original_graph, my_graph, contract, tx_bound, mode, benchmark):
    g1 = MyGraph(original_graph)
    g2 = MyGraph(my_graph)
    aver = check_graphs(g1, g2)
    if aver:
        print(f"Diff found in {contract} with tx_bound {tx_bound} and mode {mode}")
    node_difference = len(g2.nodes) - len(g1.nodes)
    edge_difference = len(g2.edges) - len(g1.edges)
    object_to_append = {
        "Contract": contract,
        "TxBound": tx_bound,
        "Mode": mode,
        "Node Difference": node_difference,
        "Edge Difference": edge_difference,
    }
    if benchmark == 1:
        diff_verisol_results_1.append(object_to_append)
    elif benchmark == 2:
        diff_verisol_results_2.append(object_to_append)

In [7]:
def diff_echidna(original_graph, my_graph, contract, test_limit, mode, option, benchmark):
    g1 = MyGraph(original_graph)
    g2 = MyGraph(my_graph)
    aver = check_graphs(g1, g2)
    if aver:
        print(f"Diff found in {contract} with test_limit {test_limit} and mode {mode}, option {option}")
    node_difference = len(g2.nodes) - len(g1.nodes)
    edge_difference = len(g2.edges) - len(g1.edges)
    object_to_append = {
        "Contract": contract,
        "Test Limit": test_limit,
        "Mode": mode,
        "Node Difference": node_difference,
        "Edge Difference": edge_difference,
        "Reduce Combinations": option
    } 
    if benchmark == 1:
        diff_echidna_results_1.append(object_to_append)
    elif benchmark == 2:
        diff_echidna_results_2.append(object_to_append)

In [9]:
def diff_echidna_graphs(contracts_to_compare, test_limits, benchmark):
    print("Running diff_echidna_graphs...")
    options = ["not_reduce_combinations", "reduce_combinations"]
    for contract_and_mode in contracts_to_compare:
        contract = contract_and_mode[0]
        mode = contract_and_mode[1]
        for option in options:
            if option == "reduce_combinations" and mode == "states":
                continue
            for test_limit in test_limits:
                dir_name = f"{contract[:-5]}_fixed" if contract.endswith("Fixed") else contract
                dir_name = dir_name.replace("Time", "T")
                graph_file_name = fix_graph_name(contract, dir_name)
                if benchmark == 1:
                    #print(f"Verisol graph found in: {VERISOL_DIR}/{dir_name}/{mode}/8/graph/{graph_file_name}_Mode.{mode}")
                    verisol_graph_8 = f"{VERISOL_DIR}/{dir_name}/{mode}/8/graph/{graph_file_name}_Mode.{mode}"
                elif benchmark == 2:
                    #print(f"Verisol graph found in: {VERISOL_DIR}/{dir_name}/{mode}/8/reduce_combinations/graph/{graph_file_name}_Mode.{mode}")
                    verisol_graph_8 = f"{VERISOL_DIR}/{dir_name}/{mode}/8/reduce_combinations/graph/{graph_file_name}_Mode.{mode}" 
                else: 
                    print("Invalid benchmark")
                    exit(1)

                #print(f"Compared with graph in: {ECHIDNA_DIR}/{dir_name}/{mode}/{test_limit}/{option}/graph/{graph_file_name}_Mode.{mode}")
                #print("\n")
                echidna_graph = f"{ECHIDNA_DIR}/{dir_name}/{mode}/{test_limit}/{option}/graph/{graph_file_name}_Mode.{mode}"

                if not os.path.isfile(echidna_graph):
                    print(f"3. File not found: {echidna_graph}")
                if not os.path.isfile(verisol_graph_8):
                    print(f"4. File not found: {verisol_graph_8}")
                if os.path.isfile(echidna_graph) and os.path.isfile(verisol_graph_8):
                    diff_echidna(verisol_graph_8, echidna_graph, contract, test_limit, mode, option, benchmark)

In [10]:
def diff_two_verisol_graphs(contract, mode, benchmark):
    dir_name = f"{contract[:-5]}_fixed" if contract.endswith("Fixed") else contract
    dir_name = dir_name.replace("Time", "T")
    graph_file_name = fix_graph_name(contract, dir_name)
    
    # Get VeriSol graphs paths
    if benchmark == 1:
        verisol_graph_8 = f"{VERISOL_DIR}/{dir_name}/{mode}/8/graph/{graph_file_name}_Mode.{mode}"        
        verisol_graph_4 = f"{VERISOL_DIR}/{dir_name}/{mode}/4/graph/{graph_file_name}_Mode.{mode}"
    elif benchmark == 2:
        verisol_graph_8 = f"{VERISOL_DIR}/{dir_name}/{mode}/8/reduce_combinations/graph/{graph_file_name}_Mode.{mode}"        
        verisol_graph_4 = f"{VERISOL_DIR}/{dir_name}/{mode}/4/reduce_combinations/graph/{graph_file_name}_Mode.{mode}"
    else:
        print("Benchmark not found")
    
    if not os.path.isfile(verisol_graph_8):
        print(f"1. File not found: {verisol_graph_8}")
    if not os.path.isfile(verisol_graph_4):
        print(f"2. File not found: {verisol_graph_4}")
    if os.path.isfile(verisol_graph_8) and os.path.isfile(verisol_graph_4): # Happy path
        print(f"Comparing graph in: {verisol_graph_8}")
        print(f"With graph in {verisol_graph_4}")
        diff_verisol(verisol_graph_8, verisol_graph_4, contract, 4, mode, benchmark)

In [11]:
def diff_verisol_graphs(contracts_to_compare, benchmark):
    print("Runing diff_verisol_graphs...")
    for contract_and_mode in contracts_to_compare:
        contract = contract_and_mode[0]
        mode = contract_and_mode[1]
        diff_two_verisol_graphs(contract, mode, benchmark)


In [12]:
diff_verisol_results_1 = []
diff_echidna_results_1 = []

diff_verisol_results_2 = []
diff_echidna_results_2 = []

In [13]:
# diff_echidna_graphs(contracts_to_compare_1, test_limits, 1)
# diff_verisol_graphs(contracts_to_compare_1, 1)

# diff_echidna_graphs(contracts_to_compare_2, test_limits, 2)
# diff_verisol_graphs(contracts_to_compare_2, 2)

In [14]:
# with open("diff_echidna_benchmark_1.json", "w", encoding='utf-8') as fileout:
#     json.dump(diff_echidna_results_1, fileout)

# with open("diff_verisol_benchmark_1.json", "w", encoding='utf-8') as fileout:
#     json.dump(diff_verisol_results_1, fileout)

# with open("diff_echidna_benchmark_2.json", "w", encoding='utf-8') as fileout:
#     json.dump(diff_echidna_results_2, fileout)

# with open("diff_verisol_benchmark_2.json", "w", encoding='utf-8') as fileout:
#     json.dump(diff_verisol_results_2, fileout)

In [12]:
def check_dummies(contracts_to_compare, test_limits, benchmark):
    print("Running diff_echidna_graphs...")
    options = ["not_reduce_combinations", "reduce_combinations"]
    for contract_and_mode in contracts_to_compare:
        contract = contract_and_mode[0]
        mode = contract_and_mode[1]
        for option in options:
            if option == "reduce_combinations" and mode == "states":
                continue
            for test_limit in test_limits:
                dir_name = f"{contract[:-5]}_fixed" if contract.endswith("Fixed") else contract
                dir_name = dir_name.replace("Time", "T")
                graph_file_name = fix_graph_name(contract, dir_name)

                echidna_graph = f"{ECHIDNA_DIR}/{dir_name}/{mode}/{test_limit}/{option}/graph/{graph_file_name}_Mode.{mode}"

                if not os.path.isfile(echidna_graph):
                    print(f"3. File not found: {echidna_graph}")
                else:
                    see_dummies(echidna_graph, contract, test_limit, mode, option, benchmark)

In [25]:
def see_dummies(my_graph, contract, test_limit, mode, option, benchmark):
    g1 = MyGraph(my_graph)
    transitions = g1.edges
    # see if any of the edges has the substring "dummy" inside
    # in that case, print the contract, test_limit, mode, option and the edge
    for transition in transitions:
        if "dummy" in transition[1]:
            #print(f"Dummy transition found in {contract} with test_limit {test_limit} and mode {mode}, option {option}")
            #print(transition[0][0] == transition[0][1])
            if (transition[0][0] != transition[0][1]):
                print("Error")

In [26]:
check_dummies(contracts_to_compare_1, test_limits, 1)
check_dummies(contracts_to_compare_2, test_limits, 2)

Running diff_echidna_graphs...
Running diff_echidna_graphs...
